In [1]:
import json
import galois
from py_ecc.optimized_bn128 import FQ
from utils import (
    GPoint,
    SRS,
    G1,
    G2,
    generator1,
    generator2,
    validate_point,
    numbers_to_hash,
    patch_galois,
)


G1 = generator1()
G2 = generator2()

patch_galois(galois.Poly)

proof = json.load(open("proof.json", "r"))
p = proof.pop("Fp")

Fp = galois.GF(p)

circuit = json.load(open("circuit.json", "r"))
circuit.pop("Fp")
for k, v in circuit.items():
    if k in ["tau", "k1", "k2", "Fp", "omega", "n"]:
        circuit[k] = Fp(v)
    elif isinstance(v, bool):
        continue
    else:
        circuit[k] = galois.Poly(coeffs=v, field=Fp)
encrypted = circuit["encrypted"]
n = int(circuit["n"])
QM = circuit["QM"]
QL = circuit["QL"]
QR = circuit["QR"]
QO = circuit["QO"]
QC = circuit["QC"]
S1 = circuit["S1"]
S2 = circuit["S2"]
S3 = circuit["S3"]
Zh = circuit["Zh"]
L1 = circuit["L1"]
PI = circuit["PI"]
tau = circuit["tau"] if not encrypted else SRS(circuit["tau"], n)
k1 = circuit["k1"]
k2 = circuit["k2"]
omega = circuit["omega"]

for k, v in proof.items():
    if isinstance(v, list):
        proof[k] = [Fp(x) for x in v]
    elif isinstance(v, str):
        proof[k] = GPoint(*[FQ(int(x)) for x in v.strip("(").strip(")").split(",")])
    else:
        proof[k] = Fp(v)

round1 = [proof["A"], proof["B"], proof["C"]]
round2 = [proof["Z"]]
round3 = [proof["Tl"], proof["Tm"], proof["Th"]]
round4 = [
    proof["a_zeta"],
    proof["b_zeta"],
    proof["c_zeta"],
    proof["s1_zeta"],
    proof["s2_zeta"],
    proof["z_omega_zeta"],
]
round5 = [proof["Wzeta"], proof["Womega_zeta"]]

OMP: Info #270: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


# Verifier

In [2]:
# These evaluations are calculated beforehand during the setup phase
qm_exp = QM(tau)
ql_exp = QL(tau)
qr_exp = QR(tau)
qo_exp = QO(tau)
qc_exp = QC(tau)
s1_exp = S1(tau)
s2_exp = S2(tau)
s3_exp = S3(tau)

# Values provided by the prover (round 1 to 5) is a proof.
a_exp = round1[0]
b_exp = round1[1]
c_exp = round1[2]

z_exp = round2[0]

tl_exp = round3[0]
tm_exp = round3[1]
th_exp = round3[2]

# Note: verifier has to verify that the following values are in the correct Fp field
a_zeta, b_zeta, c_zeta, s1_zeta, s2_zeta, z_omega_zeta = round4

w_zeta_exp = round5[0]
w_omega_zeta_exp = round5[1]

# Note: verifier has to verify that the following values are on the curve
if encrypted:
    validate_point(qm_exp)
    validate_point(ql_exp)
    validate_point(qr_exp)
    validate_point(qo_exp)
    validate_point(qc_exp)
    validate_point(z_exp)
    validate_point(s1_exp)
    validate_point(s2_exp)
    validate_point(s3_exp)
    validate_point(tl_exp)
    validate_point(tm_exp)
    validate_point(th_exp)
    validate_point(a_exp)
    validate_point(b_exp)
    validate_point(c_exp)
    validate_point(w_zeta_exp)
    validate_point(w_omega_zeta_exp)

beta = numbers_to_hash(round1 + [0], Fp)
gamma = numbers_to_hash(round1 + [1], Fp)
alpha = numbers_to_hash(round1 + round2, Fp)
zeta = numbers_to_hash(round1 + round2 + round3, Fp)

print(f"round1 = {round1}")
print(f"round2 = {round2}")
print(f"round3 = {round3}")
print(f"round4 = {round4}")

v = numbers_to_hash(round1 + round2 + round3 + round4, Fp)
u = numbers_to_hash(round1 + round2 + round3 + round4 + round5, Fp)
u = Fp(2)

print("beta", beta)
print("gamma", gamma)
print("alpha", alpha)
print("zeta", zeta)
print("v", v)
print("u", u)

round1 = [GF(233, order=241), GF(201, order=241), GF(82, order=241)]
round2 = [GF(92, order=241)]
round3 = [GF(142, order=241), GF(45, order=241), GF(34, order=241)]
round4 = [GF(56, order=241), GF(174, order=241), GF(57, order=241), GF(140, order=241), GF(217, order=241), GF(224, order=241)]
beta 119
gamma 171
alpha 64
zeta 148
v 108
u 2


In [3]:
Zh_z = Zh(zeta)
L1_z = L1(zeta)
PI_z = PI(zeta)

r0 = (
    PI_z
    - L1_z * alpha**2
    - (a_zeta + beta * s1_zeta + gamma)
    * (b_zeta + beta * s2_zeta + gamma)
    * (c_zeta + gamma)
    * z_omega_zeta
    * alpha
)

In [4]:
D_exp = (
    qm_exp * a_zeta * b_zeta
    + ql_exp * a_zeta
    + qr_exp * b_zeta
    + qo_exp * c_zeta
    + qc_exp
)

D_exp += z_exp * (
    (a_zeta + beta * zeta + gamma)
    * (b_zeta + beta * zeta * k1 + gamma)
    * (c_zeta + beta * zeta * k2 + gamma)
    * alpha
    + L1_z * alpha**2
    + u
)

D_exp -= (
    s3_exp
    * (a_zeta + beta * s1_zeta + gamma)
    * (b_zeta + beta * s2_zeta + gamma)
    * alpha
    * beta
    * z_omega_zeta
)

D_exp -= (tl_exp + tm_exp * zeta**n + th_exp * zeta ** (2 * n)) * Zh_z

In [5]:
F_exp = (
    D_exp
    + a_exp * v
    + b_exp * v**2
    + c_exp * v**3
    + s1_exp * v**4
    + s2_exp * v**5
)

print(f"F_exp = {F_exp}")

F_exp = 28


In [6]:
E_exp = (
    -r0
    + v * a_zeta
    + v**2 * b_zeta
    + v**3 * c_zeta
    + v**4 * s1_zeta
    + v**5 * s2_zeta
    + u * z_omega_zeta
)

if encrypted:
    E_exp = G1 * E_exp

print(f"E_exp = {E_exp}")

E_exp = 236


In [7]:
e1 = w_zeta_exp + w_omega_zeta_exp * u
e2 = (
    w_zeta_exp * zeta
    + w_omega_zeta_exp * (u * zeta * omega)
    + F_exp
    + (E_exp * Fp(p - 1))
)

if encrypted:
    pairing1 = tau.tau2.pair(e1)
    pairing2 = G2.pair(e2)

    print(f"pairing1 = {pairing1}")
    print(f"pairing2 = {pairing2}")

    assert pairing1 == pairing2, f"pairing1 != pairing2"
else:
    print("\n\n--- e1, e2 ---")
    print(f"e1 = {e1 * tau} = {e1} * tau")
    print(f"e2 = {e2}")
    assert e1 * tau == e2



--- e1, e2 ---
e1 = 158 = 126 * tau
e2 = 158


# Attack

In [8]:
# Malicious prover modifies the proof by a value of choice
a_zeta = Fp(1)

# dependent values has to be recalculated
round4[0] = a_zeta
v = numbers_to_hash(round1 + round2 + round3 + round4, Fp)

Zh_z = Zh(zeta)
L1_z = L1(zeta)
PI_z = PI(zeta)

r0 = (
    PI_z
    - L1_z * alpha**2
    - (a_zeta + beta * s1_zeta + gamma)
    * (b_zeta + beta * s2_zeta + gamma)
    * (c_zeta + gamma)
    * z_omega_zeta
    * alpha
)

D_exp = (
    qm_exp * a_zeta * b_zeta
    + ql_exp * a_zeta
    + qr_exp * b_zeta
    + qo_exp * c_zeta
    + qc_exp
)

D_exp += z_exp * (
    (a_zeta + beta * zeta + gamma)
    * (b_zeta + beta * zeta * k1 + gamma)
    * (c_zeta + beta * zeta * k2 + gamma)
    * alpha
    + L1_z * alpha**2
    + u
)

D_exp -= (
    s3_exp
    * (a_zeta + beta * s1_zeta + gamma)
    * (b_zeta + beta * s2_zeta + gamma)
    * alpha
    * beta
    * z_omega_zeta
)

D_exp -= (tl_exp + tm_exp * zeta**n + th_exp * zeta ** (2 * n)) * Zh_z

F_exp = (
    D_exp
    + a_exp * v
    + b_exp * v**2
    + c_exp * v**3
    + s1_exp * v**4
    + s2_exp * v**5
)

E_exp = (
    -r0
    + v * a_zeta
    + v**2 * b_zeta
    + v**3 * c_zeta
    + v**4 * s1_zeta
    + v**5 * s2_zeta
    + u * z_omega_zeta
)

if encrypted:
    E_exp = G1 * E_exp

In [9]:
import json
from sympy import symbols, Eq, solve

# Define the symbols
X, Y = symbols("X Y")

# Constants are assumed to be known and are represented as symbols for generality
A, B, C, _u, z, w = symbols("A B C u z w")

# Define the equations based on the provided system
eq1 = Eq(X + _u * Y, A)
eq2 = Eq(z * X + _u * z * w * Y + C, B)

# Solve the system of equations
solutions = solve((eq1, eq2), (X, Y))
print(solutions)

# Capture original solutions to the equation
_A = e1
_B = e2

# New adjusted solutions to the equation
_C = F_exp - E_exp

# New W(ω*zeta)
y = (_B - _C - _A * zeta) * (u * omega * zeta - u * zeta) ** (-1)

print(f"y = {y}")
print(f"w_omega_zeta_exp = {w_omega_zeta_exp}")

# New W(zeta)
x = (_A * omega * zeta - _B + _C) * (omega * zeta - zeta) ** (-1)

print(f"x = {x}")
print(f"w_zeta_exp = {w_zeta_exp}")

assert w_zeta_exp != x
assert w_omega_zeta_exp != y

e1_new = x + y * u
e2_new = x * zeta + y * (u * zeta * omega) + F_exp - E_exp

print(f"e2_new = {x * zeta} + {y * (u * zeta * omega)} + {F_exp} - {E_exp}")

if encrypted:
    pairing1 = tau.tau2.pair(e1_new)
    pairing2 = G2.pair(e2_new)

    assert pairing1 == pairing2, f"pairing1 != pairing2"
else:
    assert e1_new * tau == e2_new

{X: (A*w*z - B + C)/(w*z - z), Y: (-A*z + B - C)/(u*w*z - u*z)}
y = 181
w_omega_zeta_exp = 183
x = 5
w_zeta_exp = 1
e2_new = 17 + 51 + 14 - 165


In [10]:
from plonk_weak_verifier import verify

proof = {
    "A": round1[0],
    "B": round1[1],
    "C": round1[2],
    "Z": round2[0],
    "Tl": round3[0],
    "Tm": round3[1],
    "Th": round3[2],
    "Wzeta": x,
    "Womega_zeta": y,
    "a_zeta": a_zeta,
    "b_zeta": round4[1],
    "c_zeta": round4[2],
    "s1_zeta": round4[3],
    "s2_zeta": round4[4],
    "z_omega_zeta": round4[5],
    "Fp": p,
}

print(verify(proof, circuit, Fp, weak=True))

True
